# <center>MORTES NOS CONTINENTES CAUSADAS PELO COVID-19</center> 
#### <center>Análisado por [Jennifer Dominique](https://github.com/JenniferDominique)</center><center>Fatec - 3º ADS/ 2020</center>

---

### **<center>⚠️ Disclaimer / Aviso Legal: </center>**
Este notebook tem um fim único e exclusivamente didático, para alunos da FATEC, como uma introdução à Análise de Dados Públicos. Desse modo, está excluída a garantia ou responsabilidade de qualquer tipo, por exemplo, de precisão, confiabilidade, completude e atualidade das informações.

## <center>Análise de dados das mortes do COVID-19 🦠</center>

Não é novidade que nesse ano de 2020 estamos vivendo a pandemia do Covid-19, Coronavírus. 

Por conta de ser um assunto atual nada melhor do que explora-lo como forma de conhecimento.
Através deste notebook, analiso a partir de gráficos em um mapa interativo, a evolução das mortes por esse vírus nos continente, em cada mês, até a atual data (junho/2020).

🎲**DataFrame retirado:** 
    
   + Site: [Our World In Data](https://ourworldindata.org/coronavirus)
   + GitHub: [owid/covid-19-data](https://github.com/owid/covid-19-data)

Data de download: 10/07/2020

---

## <center>📥 Importação das bibliotecas e base de dados utilizados nesta análise</center>

### 📚 Bibliotecas:

In [1]:
import pandas as pd
# Biblioteca utilizada para a manipulação e análise dos dados

import altair as alt
# Geração de gráficos de barras no mapa

import folium
from folium import plugins
# Mapa mundi como informativo

### 🎲 Base de Dados:

In [2]:
filename = "https://raw.githubusercontent.com/JenniferDominique/Mortes_No_Mundo_Causadas_Pelo_Covid-19/master/owid-covid-data.csv"
# Repositório no GitHub onde está armazenada a base de dados

covid = pd.read_csv(filename, delimiter=',', error_bad_lines=False) # Lendo a base de dados
# covid é um DataFrame/ Base de Dados
# Separando em colunas através do delimitador que no caso é a vírgula(,)

In [3]:
covid # Exibindo o DataFrame

,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
0,AFG,Asia,Afghanistan,2019-12-31,0.0,0.0,0.0,0.0,0.0,0.0,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83
1,AFG,Asia,Afghanistan,2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83
2,AFG,Asia,Afghanistan,2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83
3,AFG,Asia,Afghanistan,2020-01-03,0.0,0.0,0.0,0.0,0.0,0.0,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83
4,AFG,Asia,Afghanistan,2020-01-04,0.0,0.0,0.0,0.0,0.0,0.0,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29376,NaN,NaN,International,2020-02-28,705.0,0.0,4.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29377,NaN,NaN,International,2020-02-29,705.0,0.0,6.0,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29378,NaN,NaN,International,2020-03-01,705.0,0.0,6.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29379,NaN,NaN,International,2020-03-02,705.0,0.0,6.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Podemos ver as informações contidas no DataFrame está em inglês, isso se deve ao fato dele ter sido retirado de um site americano.

 ### 📇 Declarando Funções Auxiliares:

Como a intenção é analisar as mortes nos continentes, logo, temos que saber quais e como os continentes estão sendo informados no DataFrame.

In [4]:
covid.continent.unique()
# Pegando do DataFrame os nomes que são citados na coluna de continentes 
# NaN - São os dados referentes ao mundo como um todo

array(['Asia', 'Europe', 'Africa', 'North America', 'South America',
       'Oceania', nan], dtype=object)

As informações desse DataFrame é referente a cada dia desde o dia 31/12/2019 até hoje, 10/07/2020, mas como o mês de Julho só está no início, ele não foi incluído na análise, para o mês de Julho não ser comparado apenas com 10 dias, enquanto os outros meses possuem os dados de uma média de 30 dias.  

Como os dados do total das mortes são acumulados, então pegando o último dia de cada mês, conseguimos saber quantas mortes tivemos naquele mês.

In [5]:
ultimoDiaMes = ["date == '2020-06-30'", "date == '2019-12-31'", "date == '2020-01-31'","date == '2020-02-29'", "date == '2020-03-31'",
         "date == '2020-04-30'", "date == '2020-05-31'", "date == '2020-06-30'"]

# A primeira data é utilizada para calcular o total de mortes até o dia 30/06/2020
# por isso ela é a última data dos dados do DataFrame já que as mortes estão acomuladas

In [6]:
meses = ['TOTAL', '*Dez/2019','1º Jan','2º Fev','3º Mar','4º Abr','5º Mai','6º Jun']

❗ **Obs:** As mortes na base de dados estão acumulas, então para saber quantas mortes teve em cada mês foi necessário subtrair as mortes dos meses anteriores.

In [7]:
def mortesContinente(continente):
    x = 0
    mortes = []
    mortePassada = 0

    while x < len(ultimoDiaMes):
        selecaoData = continente.query(ultimoDiaMes[x])
        morteMes = selecaoData["total_deaths"].sum().astype('int')
    
        if x > 1:
            mortePassada = mortePassada + mortes[x-1]
            morteMes = morteMes - mortePassada 
            # No dataframe o total das mortes está acumulado com os valores dos meses passados
            # Então estou pegando os valores dos meses passados e armazenando-os
            # E subtraindo do total de mortes do mês atual 
            # Para descobrir quantas mortes houveram naquele mês
    
        mortes.append(morteMes)
        x = x +1
    
    return mortes

Gerando um gráfico para a melhor visualização dos dados encontrados.

In [8]:
def gerarGrafico(mortesContinente, titulo):
    sw_means = pd.DataFrame({'Meses' : [meses[1],meses[2],meses[3],meses[4],meses[5],meses[6], meses[7], meses[0]],
                        'Total de Mortes' : [mortesContinente[1], mortesContinente[2], mortesContinente[3], 
                                             mortesContinente[4], mortesContinente[5], mortesContinente[6], mortesContinente[7],
                                             mortesContinente[0]]})
    # A posição 0 está no final para a coluna do total de mortes ficar por último

    graficoMortesMeses = alt.Chart(sw_means, width = 300).mark_bar().encode(
        x = 'Total de Mortes',
        y = 'Meses',
        # O destaque será definido no resultado de uma declaração condicional
        color = alt.condition(
            alt.datum.Meses == 'TOTAL',  # Se o conteúdo da lista Meses for 'TOTAL' esse teste retornará True,
            alt.value('orange'), # Que define a barra laranja
            alt.value('steelblue') # E se não for verdade, define como azul
        )).properties(title = titulo)
    
    # Adicionando os valores em frente a barra, para saber o número preciso de quantas mortes foram naquele mês
    quantidadeMortes = graficoMortesMeses.mark_text(
        align='left',
        baseline='middle',
        dx=3  # Desloca o texto para a direita, para que não apareça na parte superior da barra
        ).encode(text ='Total de Mortes')

    return (graficoMortesMeses + quantidadeMortes).properties(height = 250)

Transformando o gráfico gerado e um pop-up para posteriormente ser colocado no mapa informativo.

In [9]:
def adicionarPopUp (grafico, latitude, longitude):
    vega = folium.features.VegaLite(grafico, width = '100%', height = '100%')
    # Declarando o gráfico gerado anteriormente
    
    sw_marker = folium.features.Marker([latitude, longitude], tooltip = 'Click', 
                                       icon = folium.Icon(color = 'red', icon = 'plus', prefix ='fa'))
    # Criando um marcador
    # Quando o mouse passar em cima da imagem do pin ele aparecerá o texto 'click'
    
    sw_popup = folium.Popup()
    
    vega.add_to(sw_popup)
    # Definindo o gráfico como um popup
    
    sw_popup.add_to(sw_marker)
    # Associando o gráfico ao marcador
    
    return sw_marker

Função para criar um mapa informativo, bastando somente informa a latitude, longitude e o zoom de como o mapa será visualizado no início.

In [10]:
def criandoMapa (latitude, longitude, zoom):
    mapa = folium.Map(location = [latitude, longitude], zoom_start = zoom, control_scale = True) 
    # Localidade do mapa na imagem informando Latitude e Longitude

    minimap = plugins.MiniMap(toggle_display = True) 
    # Criando um mini mapa auxiliar de localização

    mapa.add_child(minimap) 
    # Adicionando o mini mapa no mapa

    plugins.Fullscreen(position='topright').add_to(mapa) 
    # Botão de full screen no canto superior direito

    return mapa

# <center>Analisando As Mortes Nos Continentes 🕵🏻‍♀️</center>

---

# 🐘 **Continente Africano**

---

Dentro do DataFrame estou pegando somente os dados que possuem o continente Africano.

In [11]:
africa = covid.query("continent == 'Africa'")

africa.head() # Exibindo apenas os 5 primeiros dados

,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
307,DZA,Africa,Algeria,2019-12-31,0.0,0.0,0.0,0.0,0.0,0.0,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.9,76.88
308,DZA,Africa,Algeria,2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.9,76.88
309,DZA,Africa,Algeria,2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.9,76.88
310,DZA,Africa,Algeria,2020-01-03,0.0,0.0,0.0,0.0,0.0,0.0,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.9,76.88
311,DZA,Africa,Algeria,2020-01-04,0.0,0.0,0.0,0.0,0.0,0.0,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.9,76.88


Calculando quantas mortes o continente teve em cada mês.

In [12]:
mortes_africa = mortesContinente(africa)
mortes_africa

# O primeiro valor é o total de mortes até agora

[9878, 0, 0, 0, 166, 1425, 2480, 5807]

Gerando um gráfico com os dados obtidos.

In [13]:
africa_mortesMeses = gerarGrafico(mortes_africa, '🐘 Mortes no continente Africano')
africa_mortesMeses

alt.LayerChart(...)

Agora para conseguirmos pegar os dados dos outros continentes, é só seguir a mesma ideia do que foi feito para o continente Africano.

# 🎎**Continente Asiático**

---

In [14]:
asia = covid.query("continent == 'Asia'")
asia.head()

,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
0,AFG,Asia,Afghanistan,2019-12-31,0.0,0.0,0.0,0.0,0.0,0.0,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83
1,AFG,Asia,Afghanistan,2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83
2,AFG,Asia,Afghanistan,2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83
3,AFG,Asia,Afghanistan,2020-01-03,0.0,0.0,0.0,0.0,0.0,0.0,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83
4,AFG,Asia,Afghanistan,2020-01-04,0.0,0.0,0.0,0.0,0.0,0.0,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83


In [15]:
mortes_asia = mortesContinente(asia)
mortes_asia

[55848, 0, 213, 2679, 3997, 11373, 12003, 25583]

In [16]:
asia_mortesMeses = gerarGrafico(mortes_asia, '🎎 Mortes no continente Asiatico')
asia_mortesMeses

alt.LayerChart(...)

# 💂🏻‍♂️ **Continente Europeu**

---

In [17]:
europa = covid.query("continent == 'Europe'")
europa.head()

,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
183,ALB,Europe,Albania,2020-03-09,2.0,2.0,0.0,0.0,0.695,0.695,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
184,ALB,Europe,Albania,2020-03-10,6.0,4.0,0.0,0.0,2.085,1.390,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
185,ALB,Europe,Albania,2020-03-11,10.0,4.0,0.0,0.0,3.475,1.390,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
186,ALB,Europe,Albania,2020-03-12,11.0,1.0,1.0,1.0,3.822,0.347,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
187,ALB,Europe,Albania,2020-03-13,23.0,12.0,1.0,0.0,7.992,4.170,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57


In [18]:
mortes_europa = mortesContinente(europa)
mortes_europa

[190771, 0, 0, 23, 27912, 105319, 39751, 17766]

In [19]:
europa_mortesMeses = gerarGrafico(mortes_europa, '💂🏻‍♂️ Mortes no continente Europeu')
europa_mortesMeses

alt.LayerChart(...)

# 🗽 **Continentes Central e Norte Americano**

---

In [20]:
norteAmericano = covid.query("continent == 'North America'")
norteAmericano.head()

,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
725,AIA,North America,Anguilla,2020-03-27,2.0,2.0,0.0,0.0,133.316,133.316,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81.88
726,AIA,North America,Anguilla,2020-03-28,2.0,0.0,0.0,0.0,133.316,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81.88
727,AIA,North America,Anguilla,2020-03-29,2.0,0.0,0.0,0.0,133.316,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81.88
728,AIA,North America,Anguilla,2020-03-30,2.0,0.0,0.0,0.0,133.316,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81.88
729,AIA,North America,Anguilla,2020-03-31,2.0,0.0,0.0,0.0,133.316,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81.88


In [21]:
mortes_norteAmericano = mortesContinente(norteAmericano)
mortes_norteAmericano

[165087, 0, 0, 0, 3381, 63108, 55701, 42897]

In [22]:
norteAmericano_mortesMeses = gerarGrafico(mortes_norteAmericano, '🗽 Mortes nos continentes Central e Norte Americano')
norteAmericano_mortesMeses

alt.LayerChart(...)

# 🦘 **Continente Oceanico**

---

In [23]:
oceania = covid.query("continent == 'Oceania'")
oceania.head()

,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
1364,AUS,Oceania,Australia,2019-12-31,0.0,0.0,0.0,0.0,0.0,0.0,...,10.129,44648.71,0.5,107.791,5.07,13.0,16.5,NaN,3.84,83.44
1365,AUS,Oceania,Australia,2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0,...,10.129,44648.71,0.5,107.791,5.07,13.0,16.5,NaN,3.84,83.44
1366,AUS,Oceania,Australia,2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0,...,10.129,44648.71,0.5,107.791,5.07,13.0,16.5,NaN,3.84,83.44
1367,AUS,Oceania,Australia,2020-01-03,0.0,0.0,0.0,0.0,0.0,0.0,...,10.129,44648.71,0.5,107.791,5.07,13.0,16.5,NaN,3.84,83.44
1368,AUS,Oceania,Australia,2020-01-04,0.0,0.0,0.0,0.0,0.0,0.0,...,10.129,44648.71,0.5,107.791,5.07,13.0,16.5,NaN,3.84,83.44


In [24]:
mortes_oceania = mortesContinente(oceania)
mortes_oceania

[133, 0, 0, 0, 21, 95, 15, 2]

In [25]:
oceania_mortesMeses = gerarGrafico(mortes_oceania, '🦘 Mortes no continente Oceanico')
oceania_mortesMeses

alt.LayerChart(...)

# 🗿 **Continente Sul Americano**

---

In [26]:
sulAmericano = covid.query("continent == 'South America'")
sulAmericano.head()

,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
944,ARG,South America,Argentina,2020-03-04,1.0,1.0,0.0,0.0,0.022,0.022,...,7.441,18933.907,0.6,191.032,5.5,16.2,27.7,NaN,5.0,76.67
945,ARG,South America,Argentina,2020-03-06,2.0,1.0,0.0,0.0,0.044,0.022,...,7.441,18933.907,0.6,191.032,5.5,16.2,27.7,NaN,5.0,76.67
946,ARG,South America,Argentina,2020-03-07,8.0,6.0,0.0,0.0,0.177,0.133,...,7.441,18933.907,0.6,191.032,5.5,16.2,27.7,NaN,5.0,76.67
947,ARG,South America,Argentina,2020-03-08,9.0,1.0,1.0,1.0,0.199,0.022,...,7.441,18933.907,0.6,191.032,5.5,16.2,27.7,NaN,5.0,76.67
948,ARG,South America,Argentina,2020-03-09,12.0,3.0,1.0,0.0,0.266,0.066,...,7.441,18933.907,0.6,191.032,5.5,16.2,27.7,NaN,5.0,76.67


In [27]:
mortes_sulAmericano = mortesContinente(sulAmericano)
mortes_sulAmericano

[83585, 0, 0, 0, 305, 7797, 31222, 44261]

In [28]:
sulAmericano_mortesMeses = gerarGrafico(mortes_sulAmericano, '🗿 Mortes no continente Sul Americano')
sulAmericano_mortesMeses

alt.LayerChart(...)

# 🌎 Mortes no Mundo

---

In [29]:
mundo = covid.query("location == 'World'")
mundo.tail()

,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
29312,OWID_WRL,NaN,World,2020-07-06,11419719.0,178979.0,533961.0,3380.0,1465.043,22.961,...,5.355,15469.207,10.0,233.07,8.51,6.434,34.635,60.13,2.705,72.58
29313,OWID_WRL,NaN,World,2020-07-07,11595164.0,175445.0,537815.0,3854.0,1487.551,22.508,...,5.355,15469.207,10.0,233.07,8.51,6.434,34.635,60.13,2.705,72.58
29314,OWID_WRL,NaN,World,2020-07-08,11802188.0,207024.0,543906.0,6091.0,1514.111,26.559,...,5.355,15469.207,10.0,233.07,8.51,6.434,34.635,60.13,2.705,72.58
29315,OWID_WRL,NaN,World,2020-07-09,12017661.0,215473.0,549281.0,5375.0,1541.754,27.643,...,5.355,15469.207,10.0,233.07,8.51,6.434,34.635,60.13,2.705,72.58
29316,OWID_WRL,NaN,World,2020-07-10,12245417.0,227756.0,554721.0,5440.0,1570.973,29.219,...,5.355,15469.207,10.0,233.07,8.51,6.434,34.635,60.13,2.705,72.58


In [30]:
mundo.query("date == '2020-07-10'")

,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
29316,OWID_WRL,NaN,World,2020-07-10,12245417.0,227756.0,554721.0,5440.0,1570.973,29.219,...,5.355,15469.207,10.0,233.07,8.51,6.434,34.635,60.13,2.705,72.58


In [31]:
mortes_mundo = mortesContinente(mundo)
mortes_mundo

[505309, 0, 213, 2708, 35784, 189116, 141172, 136316]

In [32]:
mundo_mortesMeses = gerarGrafico(mortes_mundo, '🌎 Mortes no Mundo')
mundo_mortesMeses

alt.LayerChart(...)

# <center>🗺 Mapa Interativo</center>

---

In [33]:
map_mundi = criandoMapa(20, 40, 2) # Criando uma mapa

#____ Adicionando as informações no mapa ____

adicionarPopUp(mundo_mortesMeses, 22.59, -33.75).add_to(map_mundi)
# Adicionando os dados do mundo no mapa

adicionarPopUp(africa_mortesMeses, 5.26, 23.55).add_to(map_mundi) 
# Adicionando os dados da Africa no mapa

adicionarPopUp(asia_mortesMeses, 49.09, 88.24).add_to(map_mundi)
# Adicionando os dados da Asia no mapa

adicionarPopUp(europa_mortesMeses, 49.26, 21.44).add_to(map_mundi)
# Adicionando os dados da Europa no mapa

adicionarPopUp(norteAmericano_mortesMeses, 48.99, -106.52).add_to(map_mundi)
# Adicionando os dados da America do Norte no mapa

adicionarPopUp(oceania_mortesMeses,-24.52, 136.05).add_to(map_mundi)
# Adicionando os dados da Oceania no mapa

adicionarPopUp(sulAmericano_mortesMeses,-16.72, -59.76).add_to(map_mundi)
# Adicionando os dados da América do Sul no mapa


map_mundi # Exibindo o mapa
# Visualizar o mapa com os dados dos continentes